<a href="https://colab.research.google.com/github/VeraMikhalchenko/Study/blob/main/ML_HW2_Logloss_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Домашнее задание «Функции потерь и оптимизация»**

1. Загрузите данные. Используйте датасет с ирисами. Его можно загрузить непосредственно из библиотеки Sklearn. В данных оставьте только 2 класса: Iris Versicolor, Iris Virginica.

In [255]:
from sklearn import datasets
import pandas as pd
import numpy as np
import math
import time

In [256]:
iris = datasets.load_iris()
type(iris)

sklearn.utils._bunch.Bunch

In [257]:
iris_df = pd.DataFrame(data=iris.data, 
                  columns=iris.feature_names)
iris_df['target'] = iris.target

iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [258]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [259]:
iris_df['name'] = iris_df.target.apply(lambda x : iris.target_names[x])

Оставляем только два класса

In [260]:
iris_df = iris_df[iris_df['target'] != 0]
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,name
50,7.0,3.2,4.7,1.4,1,versicolor
51,6.4,3.2,4.5,1.5,1,versicolor
52,6.9,3.1,4.9,1.5,1,versicolor
53,5.5,2.3,4.0,1.3,1,versicolor
54,6.5,2.8,4.6,1.5,1,versicolor


In [261]:
iris_df['name'].value_counts()

versicolor    50
virginica     50
Name: name, dtype: int64

In [262]:
iris_df.reset_index(drop=True, inplace=True)
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,name
0,7.0,3.2,4.7,1.4,1,versicolor
1,6.4,3.2,4.5,1.5,1,versicolor
2,6.9,3.1,4.9,1.5,1,versicolor
3,5.5,2.3,4.0,1.3,1,versicolor
4,6.5,2.8,4.6,1.5,1,versicolor


In [263]:
iris_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sepal length (cm)  100 non-null    float64
 1   sepal width (cm)   100 non-null    float64
 2   petal length (cm)  100 non-null    float64
 3   petal width (cm)   100 non-null    float64
 4   target             100 non-null    int64  
 5   name               100 non-null    object 
dtypes: float64(4), int64(1), object(1)
memory usage: 4.8+ KB


2. **Самостоятельно реализуйте логистическую регрессию, без использования метода LogisticRegression из библиотеки. Можете использовать библиотеки pandas, numpy, math для реализации. Оформите в виде функции. *Оформите в виде класса с методами.**

3. **Реализуйте метод градиентного спуска. Обучите логистическую регрессию этим методом. Выберете и посчитайте метрику качества. Метрика должна быть одинакова для всех пунктов домашнего задания. Для упрощения сравнения выберете только одну метрику.**

Напишем функцию для оценки модели с помощью метрики Accuracy

In [264]:
def acc_metric(y_pred, y_test):
    '''Функция подсчитывает метрику Accuracy'''

    k = y_test.shape
    y_error = np.abs(y_test - y_pred)
    acc = (1 - np.sum(y_error)/k) * 100

    return acc[0]

In [265]:
from sklearn.model_selection import train_test_split

In [266]:
Х = iris_df[['sepal length (cm)','sepal width (cm)','petal length (cm)','petal width (cm)']]

In [267]:
Х.reset_index(drop=True, inplace=True)
Х.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,7.0,3.2,4.7,1.4
1,6.4,3.2,4.5,1.5
2,6.9,3.1,4.9,1.5
3,5.5,2.3,4.0,1.3
4,6.5,2.8,4.6,1.5


In [268]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [269]:
le.fit(iris_df['target'])

LabelEncoder()

In [270]:
le.classes_

array([1, 2])

In [271]:
le.transform([1, 2, 1])

array([0, 1, 0])

In [272]:
y = pd.Series (data = le.transform(iris_df['target']))
y.head()

0    0
1    0
2    0
3    0
4    0
dtype: int64

In [273]:
Х_train, Х_test, y_train, y_test = train_test_split(Х, y, test_size = 0.3, random_state = 42)

In [274]:
class LogRegSimple:
    '''Базовый класс LogReg реализует логистическую регрессию для бинарных классов
    с помощью обычного градиентного спуска..
    В качестве входных параметров принимает: 
    - шаг (learning rate)
    - дополнительный критерий останова (tol) (будет реализован в подклассе)
    - максимальное количество итераций (max_iterations)'''

    def __init__(self, learning_rate, tol, max_iterations):        
        self.learning_rate = learning_rate
        self.tol = tol # размер ошибки, при которой останавливаем обучение
        self.max_iterations = max_iterations
          
    def fit(self, Х_train, y_train):
        '''Метод для обучения модели''' 
       
        # размерность
        m, n = Х_train.shape      
        
        # начальные веса       
        self.W = np.zeros(n)        
        self.b = 0 
        
        self.error = 0
        
        iterations = 1
        exit = False 

        # градиентный спуск                       
        while not exit:    
            self.update_weights(Х_train, y_train)            
            exit = self.check_stop(Х_train, y_train, iterations)
            iterations += 1

        print('iterations = ', iterations)

    def check_stop(self, Х_train, y_train, iterations):
        '''Критерий останова в виде максимального количества итераций'''

        if iterations >= self.max_iterations:
            return True

        return False
  
    def update_weights(self, X_train, y_train):
        '''Вспомогательный метод для обновления весов в градиентном спуске'''
        
        m, n = Х_train.shape  
        # вычисление градиента        
        A = 1/(1 + np.exp(-(Х_train.dot(self.W) + self.b)))
        tmp = (A - y_train.T)        
        tmp = np.reshape(tmp, m)        
        dW = np.dot(Х_train.T, tmp)/m  # производная (A - y_i) * x_i   
        db = np.sum(tmp)/m  # производная (A - y_i)
        self.update_lr() # даст возможность в подклассе реализовать динамический шаг

        # обновление весов   
        self.W -= self.learning_rate * dW    
        self.b -= self.learning_rate * db
    
    def update_lr(self):
        '''Метод в подклассе будет реализовывать динамический шаг'''
        
        pass

    def predict(self, Х_test) : 
        '''Метод предсказания'''   

        Z = 1 / (1 + np.exp(-(Х_test.dot(self.W) + self.b)))        
        y_pred = np.where(Z > 0.5, 1, 0)

        return y_pred

In [275]:
class Tol(LogRegSimple):
    '''Подкласс Tol реализует дополнительное условие останова
    в виде процента ошибочных предсказаний на тренировочной выборке'''

    def check_stop(self, Х_train, y_train, iterations):
        '''Критерий останова'''

        if super(Tol, self).check_stop(Х_train, y_train, iterations):
            return True

        m, n = Х_train.shape
        Z = 1 / (1 + np.exp(-(Х_train.dot(self.W) + self.b)))        
        y_pred_train = np.where(Z > 0.5, 1, 0)
        
        y_error = np.abs(y_train - y_pred_train)
        error = np.sum(y_error)/m
        
        return error < self.tol

In [276]:
class LogRegLr(Tol):
    '''Подкласс LogRegLr реализует более гибкий шаг в зависимости от 
    количества ошибочных предсказаний на тренировочной выборке'''
    
    def update_lr(self):
        '''Метод подобран эмпирическим путём'''
        
        if self.error < 0.1:
            self.learning_rate *= 1.1

In [277]:
start = time.time()
model_sgd = LogRegLr(learning_rate = 0.1, tol = 0.045, max_iterations = 100)
model_sgd.fit(Х_train, y_train)
end = time.time()
t_sgd = round((end - start), 3)

iterations =  40


In [278]:
y_pred = model_sgd.predict(Х_test)

In [279]:
df_compare = pd.DataFrame ([y_test.values, y_pred]).T
df_compare.columns = ['target', 'predict']
df_compare

,target,predict
0,1,1
1,1,1
2,1,1
3,0,0
4,0,0
5,0,0
6,0,1
7,1,1
8,0,0
9,0,0


In [280]:
accuracy_sgd = round(acc_metric(y_pred, y_test.values), 2)
accuracy_sgd

93.33

**4. Повторите п. 3 для метода скользящего среднего (Root Mean Square Propagation, RMSProp).**

In [281]:
class RMSProp(LogRegLr):
    '''Подкласс RMSProp (Root Mean Square Propagation) реализует 
    логистическую регрессию для бинарных классов с помощью 
    градиентного спуска методом скользящего среднего.'''

    def fit(self, Х_train, y_train):
        '''Метод для обучения модели c помощью градиентного спуска RMSProp'''
        
        # размерность
        m, n = Х_train.shape
        
        # начальные веса
        self.W = np.ones(n)
        self.b = 1
        self.v_w = np.zeros(n)
        self.v_b = 0
         
        iterations = 0
        exit = False 

        # градиентный спуск RMSProp                    
        while not exit:    
            self.update_weights(Х_train, y_train)            
            exit = self.check_stop(Х_train, y_train, iterations)
            iterations += 1

        print('iterations = ', iterations)
        
    def update_weights(self, Х_train, y_train):
        '''Вспомогательный метод для обновления весов в градиентном спуске RMSProp'''  
        
        m, n = Х_train.shape

        # гиперпараметры
        beta = 0.9
        eps = 1e-8

        # вычисление градиента 
        A = 1/(1 + np.exp(-(Х_train.dot(self.W) + self.b)))       
        tmp = (A - y_train.T)        
        tmp = np.reshape(tmp, m)        
        dW = np.dot(Х_train.T, tmp)/m         
        db = np.sum(tmp)/m 

        # обновление весов 
        self.v_w = beta * self.v_w + (1 - beta) * dW**2
        self.v_b = beta * self.v_b + (1 - beta) * db**2

        self.W -= (self.learning_rate/np.sqrt(self.v_w + eps)) * dW
        self.b -= (self.learning_rate/np.sqrt(self.v_b + eps)) * db



In [282]:
start = time.time()
model_rmsprop = RMSProp(learning_rate = 0.1, tol = 0.05, max_iterations = 200)
model_rmsprop.fit(Х_train, y_train)
end = time.time()
t_rms = round((end - start), 3)

iterations =  173


In [283]:
y_pred = model_rmsprop.predict(Х_test)

In [284]:
accuracy_rmsprop = round(acc_metric(y_pred, y_test.values), 2)
accuracy_rmsprop

86.67

**5. Повторите п. 3 для ускоренного по Нестерову метода адаптивной оценки моментов (Nesterov–accelerated Adaptive Moment Estimation, Nadam).**

In [285]:
class Nadam(LogRegLr):
    '''Подкласс Nadam (Nesterov–accelerated Adaptive Moment Estimation) 
    реализует логистическую регрессию для бинарных классов с помощью 
    градиентного спуска для ускоренного по Нестерову метода адаптивной оценки моментов.'''
    
    def fit(self, Х_train, y_train):
        '''Метод для обучения модели c помощью градиентного спуска 
        Nesterov–accelerated Adaptive Moment Estimation,'''
        
        # размерность
        m, n = Х_train.shape

        # начальные веса
        self.W = np.ones(n)
        self.b = 1      
        self.prev_v_w = np.zeros(n)
        self.prev_v_b = 0 

        iterations = 0
        exit = False 

        # градиентный спуск Nadam
        while not exit:    
            self.update_weights(Х_train, y_train)            
            exit = self.check_stop(Х_train, y_train, iterations)
            iterations += 1

        print('iterations = ', iterations)


    def update_weights(self, Х_train, y_train):
        '''Вспомогательный метод для обновления весов в градиентном спуске Nadam'''  

        m, n = Х_train.shape

        # гиперпараметр
        gamma = 0.9

        # частичное обновление
        v_w = gamma * self.prev_v_w
        v_b = gamma * self.prev_v_b 
        
        # вычисление градиента     
        A = 1/(1 + np.exp(-(Х_train.dot(self.W - v_w) + (self.b - v_b))))   
        tmp = (A - y_train.T)        
        tmp = np.reshape(tmp, m)        
        dW = np.dot(Х_train.T, tmp)/m         
        db = np.sum(tmp)/m   
        
        # обновление весов    
        v_w = gamma * self.prev_v_w + self.learning_rate * dW
        v_b = gamma * self.prev_v_b + self.learning_rate * db
        self.W -= v_w
        self.b -= v_b
        self.prev_v_w = v_w
        self.prev_v_b = v_b

    

In [286]:
start = time.time()
model_nadam = Nadam(learning_rate = 0.1, tol = 0.05, max_iterations = 200)
model_nadam.fit(Х_train, y_train)
end = time.time()
t_nad = round((end - start), 3)

iterations =  24


In [287]:
y_pred = model_nadam.predict(Х_test)

In [288]:
accuracy_nadam = round(acc_metric(y_pred, y_test.values), 2)
accuracy_nadam

93.33

**Сравните значение метрик для реализованных методов оптимизации. Можно оформить в виде таблицы вида |метод|метрика|время работы| (время работы опционально). Напишите вывод.**

In [289]:
data = {'method': ['SGD', 'RMSProp', 'Nadam'], 'accuracy': [accuracy_sgd, accuracy_rmsprop, accuracy_nadam],'time': [t_sgd,t_rms,t_nad]} 
 
df_compare_methods_grad = pd.DataFrame(data)

df_compare_methods_grad

,method,accuracy,time
0,SGD,93.33,0.090
1,RMSProp,86.67,0.334
2,Nadam,93.33,0.046


Самое точное предсказание согласно метрике accuracy у обычного градиентного спуска и у Nadam. По времени самый долгий метод RMSProp. Для нашего дататсета лучше всего использовать обычный градиентный спуск. Самое маленькое количество итераций у Nadam.

https://habr.com/ru/post/318970/ \
https://towardsdatascience.com/learning-parameters-part-1-eb3e8bb9ffbb \
https://towardsdatascience.com/learning-parameters-part-2-a190bef2d12 \
https://towardsdatascience.com/learning-parameters-part-3-ee8558f65dd7 \
https://towardsdatascience.com/learning-parameters-part-4-6a18d1d3000b \
https://towardsdatascience.com/learning-parameters-part-5-65a2f3583f7d